In [10]:
from generate import simulate_blackwell_glynn
from nsmm import nsmm_lag1

In [11]:
df = simulate_blackwell_glynn(n_units=1000, t_periods=20)
nsmm_lag1(df, unit = 'i' , treatment = 'X',outcome = 'Y',common_causes= ['Z'])

(-0.09138011216868848, -0.10421430900214641)